#**Case 3**

ÖDEV<br>
Spam verisetinde öncelikle gerekli veri ön işleme adımları yapılarak metin temizlenecek,

sonraki adımda makine öğrenmesi modelleri kullanılarak bir mailin spam olup olmadığı kurulan model ile tahmin edilecek.

# Dataset Options

In [ ]:
# Import Dataset

import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/spam.csv', encoding='ISO-8859-1',usecols=[0,1])
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# Rename Columns

df = df.rename(columns={'v1': 'label', 'v2': 'text'})
df.head()

,label,text,text_clean
0,ham,"Go until jurong point, crazy.. Available only ...","[Go, until, jurong, point, ,, crazy, .., Avail..."
1,ham,Ok lar... Joking wif u oni...,"[Ok, lar, ..., Joking, wif, u, oni, ...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[Free, entry, in, 2, a, wkly, comp, to, win, F..."
3,ham,U dun say so early hor... U c already then say...,"[U, dun, say, so, early, hor, ..., U, c, alrea..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[Nah, I, do, n't, think, he, goes, to, usf, ,,..."


In [ ]:
def review_df(df):
  print("Shape")
  print(df.shape)
  print("-"*100)
  print("Info")
  print(df.info())
  print("-"*100)
  print("Null")
  print(df.isnull().sum())
  print("-"*100)
  print("Head")
  print(df.head())
  print("-"*100)
  print("Tail")
  print(df.tail())
  print("-"*100)

review_df(df)

Shape
(5572, 3)
----------------------------------------------------------------------------------------------------
Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   label       5572 non-null   object
 1   text        5572 non-null   object
 2   text_clean  5572 non-null   object
dtypes: object(3)
memory usage: 130.7+ KB
None
----------------------------------------------------------------------------------------------------
Null
label         0
text          0
text_clean    0
dtype: int64
----------------------------------------------------------------------------------------------------
Head
  label                                               text  \
0   ham  Go until jurong point, crazy.. Available only ...   
1   ham                      Ok lar... Joking wif u oni...   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3   

#**Preprocessing**

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocessing_pipeline(df, new_column, source_column):

    print("1. Lowercase")
    df[new_column] = df[source_column].str.lower()

    print("2. Remove punctuation and digits")
    df[new_column] = df[new_column].str.replace('[^\w\s]', '', regex=True)
    df[new_column] = df[new_column].str.replace('\d+', '', regex=True)

    print("3. Remove stopwords")
    stop_words = set(stopwords.words('english'))
    df[new_column] = df[new_column].apply(lambda x: " ".join(
        word for word in x.split() if word not in stop_words))

    print("4. Strip whitespaces")
    df[new_column] = df[new_column].str.strip()

    print("5. Tokenization")
    df[new_column] = df[new_column].apply(lambda x: word_tokenize(x))

    print("Preprocessing complete!")
    return df


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [1]:
df = preprocessing_pipeline(df, new_column='normalized_text', source_column='text')

NameError: name 'preprocessing_pipeline' is not defined

In [ ]:
df['normalized_text'][0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [ ]:
df['text'][0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [ ]:
#Label Mapping

label_mapping = {'ham': 0, 'spam': 1}
df['label'] = df['label'].map(label_mapping)

In [ ]:
df['label'].head(15)

,label
0,0
1,0
2,1
3,0
4,0
5,1
6,0
7,0
8,1
9,1


#**Modelling**

In [ ]:
# Vectorize Text for train to machine

from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()

X = df['normalized_text'].apply(lambda x: ' '.join(x))
y = df['label']


X_tfidf = vectorizer.fit_transform(X)


In [ ]:
# Train Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


print("Model Score: ", model.score(X_test, y_test))
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_report(y_test, y_pred))

Model Score:  0.947085201793722
Confusion Matrix:
[[961   4]
 [ 55  95]]
Classification Report:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       0.96      0.63      0.76       150

    accuracy                           0.95      1115
   macro avg       0.95      0.81      0.87      1115
weighted avg       0.95      0.95      0.94      1115



In [ ]:
test_text = "There is a free prize if you share your account informations with we."

model.predict(vectorizer.transform([test_text]))

array([1])